## My bicycle reinvention in module import issue  

In [1]:
from lib import *

Pipeline = pipeline.Pipeline
RegressionValidator = pipeline.RegressionValidator

DatasetProvider = provider.DatasetProvider
DatasetUploader = provider.DatasetUploader
ExpandedWindowIterator = subset_extraction.ExpandedWindowIterator
EntityIterator = subset_extraction.EntityIterator


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import scipy
from scipy import stats

import warnings
from sklearn.exceptions import ConvergenceWarning, DataConversionWarning

warnings.filterwarnings('ignore', category=DataConversionWarning)
warnings.filterwarnings('ignore', category=ConvergenceWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
from sklearn.preprocessing import LabelEncoder

from statsmodels.tsa.stattools import acf 

tr = transformers

In [3]:
CONFIG = {
    'validation_window': {
        'min_window': 13,
        'max_window': 32,
    },
    'training_window': {
        'min': 16
    },
    'FETCH_DATA': True,
}

In [4]:
df_provider = DatasetProvider()

In [5]:
dataset_paths = [
    '/home/denissimo/Repo/fs_project/datasets/sample_submission.csv',
    '/home/denissimo/Repo/fs_project/datasets/test.csv',
    '/home/denissimo/Repo/fs_project/datasets/shops.csv',
    '/home/denissimo/Repo/fs_project/datasets/item_categories.csv',
    '/home/denissimo/Repo/fs_project/datasets/sales_train.csv',
    '/home/denissimo/Repo/fs_project/datasets/items.csv'
]
fetched = [
    
]

In [6]:
import pickle

def load_pickle(dataset, name):
    dataset.to_pickle(name + '.pkl')

def from_pickle(path):
    with open(path, 'rb') as reader:
        return pickle.load(reader)

In [7]:
df_provider.file_list = dataset_paths

datasets = df_provider.get_dataset()

In [8]:
sales = datasets['sales_train.csv']
items = datasets['items.csv']
categories = datasets['item_categories.csv']
shops = datasets['shops.csv']

In [9]:
force_category = {
    'category': {
        "PC - Гарнитуры/Наушники": "Аксессуары",
        "Игры MAC - Цифра": "Игры",
        "Игры Android - Цифра": "Игры",
        "Чистые носители (шпиль)": "Чистые носители",
        "Чистые носители (штучные)": "Чистые носители",
    },
    'shop': {
        'Интернет-магазин ЧС': 'Интернет-магазин',
        'Цифровой склад 1С-Онлайн': 'Склад',
        'Выездная Торговля': 'Выездная Торговля',
        '!Якутск Орджоникидзе, 56 фран': 'Якутск',
        '!Якутск ТЦ "Центральный" фран': 'Якутск',
    },
}

pattern = {
    'category': ' - ',
    'shop': ' ',
}

In [10]:
sales.date = sales.date.astype('datetime64[ns]')

print("Before:", sales.shape)

from datetime import date

sales = sales.loc[sales.date < np.datetime64(date(2015, 11, 1))]

sales_train = sales[
    (sales["item_cnt_day"] < 1000)
    & (sales["item_price"] > 0)
    & (sales["item_price"] < 60000)
].copy()
print("After:", sales_train.shape)

sales.head()

Before: (2935849, 6)
After: (2896778, 6)


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-02-01,0,59,22154,999.00,1.0
1,2013-03-01,0,25,2552,899.00,1.0
2,2013-05-01,0,25,2552,899.00,-1.0
3,2013-06-01,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0


In [11]:
make_cat_name = tr.create_name_transformer(force_category['category'], pattern['category'])
make_city_name = tr.create_name_transformer(force_category['shop'], pattern['shop'])

In [12]:
data_preprocessing = {}

# Add corresponding category and shop id's to each sale
data_preprocessing['id_merging_stage'] = lambda dataset: dataset.merge(
    items, 
    on='item_id'
).merge(
    shops,
    on='shop_id'
).merge(
    categories,
    on='item_category_id'
)

# Add summary among shop_id and category_id above similar time periods (daily intervals)
data_preprocessing['summarizing_and_name_merging_stage'] = lambda dataset: dataset.groupby(
    ['date', 'date_block_num', 'shop_id', 'item_category_id', 'item_category_name', 'shop_name']
).item_cnt_day.sum().reset_index().sort_values('date')

data_preprocessing['add_generalized_names_and_encode_stage'] = lambda dataset: tr.append_columns(
    dataset=dataset, 
    columns=[
        'global_item_category_name',
        'city_name',
        'global_item_category_name_id',
        'city_id',
    ], 
    transformers=[
        lambda _dataset: _dataset["item_category_name"].apply(
            make_cat_name
        ),
        lambda _dataset: _dataset['shop_name'].apply(
            make_city_name
        ),
        lambda _dataset: LabelEncoder().fit_transform(_dataset['global_item_category_name']),
        lambda _dataset: LabelEncoder().fit_transform(_dataset['city_name']),
    ]
)

        
data_preprocessing['date_block_num_renaming'] = lambda dataset: dataset.rename(columns={'date_block_num': 'month_block'}, inplace=True)
        
data_preprocessing['date_encoding_stage'] = lambda dataset: tr.append_columns(
    dataset=dataset, 
    columns=[
        'week_block',
        'day_block',
    ], 
    transformers=[
        lambda _dataset: LabelEncoder().fit_transform(_dataset['date'].dt.to_period('W')),
        lambda _dataset: LabelEncoder().fit_transform(_dataset['date'].dt.to_period('D')),
    ]
)

data_preprocessing['create_full_matrix_stage'] = lambda _dataset: _dataset.set_index('date') \
    .groupby([
        'shop_id',
        'item_category_id',
        'month_block',
        'city_id',
        'global_item_category_name_id',
        'id'
    ]).item_cnt_day.sum() \
    .reset_index().rename(columns={'item_cnt_day': 'item_cnt_month'}) \
    .groupby(['shop_id', 'item_category_id', 'month_block', 'city_id', 'global_item_category_name_id', 'id']).item_cnt_month.sum().reset_index() \
    .groupby(['month_block', 'id']).item_cnt_month.sum().unstack().fillna(0) \
    .stack().reset_index().rename(columns={0:'item_cnt_month'})

In [13]:
pipeline = Pipeline(
    tasks=data_preprocessing, 
    task_queue = [
        'id_merging_stage',
        'summarizing_and_name_merging_stage',
        'add_generalized_names_and_encode_stage',
        'date_block_num_renaming',
        'date_encoding_stage',
    ]
)

In [14]:
pipeline_test = pipeline(sales)
pipeline_train = pipeline(sales_train)

if not CONFIG['FETCH_DATA']:
    _ = pipeline_test.proceed_all()
    _ = pipeline_train.proceed_all()

In [15]:
task_df = {}
if CONFIG['FETCH_DATA'] == False:
    task_df['test'] = pipeline_test.result_storage['date_encoding_stage']
    task_df['train'] = pipeline_train.result_storage['date_encoding_stage']
    load_pickle(task_df['test'], 'task_df_test')
    load_pickle(task_df['train'], 'task_df_train')
else:
    task_df['test'] = from_pickle('task_df_test.pkl')
    task_df['train'] = from_pickle('task_df_train.pkl')

In [16]:
idx = task_df['test'].loc[:,['city_id', 'global_item_category_name_id', 'city_name', 'global_item_category_name']].value_counts().sort_index()
idx = pd.DataFrame({'id': [i for i in range(idx.size)]}, idx.index)
idx.reset_index(inplace=True)

In [17]:
task_df['test'] = task_df['test'].merge(idx, on=['city_id', 'global_item_category_name_id', 'city_name', 'global_item_category_name'])
task_df['train'] = task_df['train'].merge(idx, on=['city_id', 'global_item_category_name_id', 'city_name', 'global_item_category_name'])

In [18]:
idx['pair_name'] = idx['city_name'] + ' - ' + idx['global_item_category_name']

In [19]:
task_df_copy = {}

task_df_copy['train'] = task_df['train'].copy()

task_df_copy['test'] = task_df['test'].copy()
task_df['test']['pair_name'] = task_df['test']['city_name'] + ' - ' + task_df['test']['global_item_category_name']

task_df['train'] = data_preprocessing['create_full_matrix_stage'](task_df['train'])
task_df['test'] = data_preprocessing['create_full_matrix_stage'](task_df['test'])

In [20]:
task_df['train']

,month_block,id,item_cnt_month
0,0,0,66.0
1,0,1,0.0
2,0,2,20.0
3,0,3,240.0
4,0,4,263.0
...,...,...,...
14887,33,433,87.0
14888,33,434,98.0
14889,33,435,14.0
14890,33,436,12.0


For future

In [21]:
def add_aggregation(feature_set, series, name, funcs=[np.mean, np.std]):
    for func in funcs:
        feature_set[name + f'_{func.__name__}'] = func(series)

def target_extractor(month):
    return task_df['train'][task_df['train'].month_block == month + 1][['id', 'item_cnt_month']].set_index('id').sort_index()

In [22]:
iteration_rule = ExpandedWindowIterator('month_block', target_extractor, min_idx=CONFIG['validation_window']['min_window'], step=1, max_idx=CONFIG['validation_window']['max_window'])

In [23]:
subset_extractor = EntityIterator('id')

In [24]:
DAYS = [31,28,31,30,31,30,31,31,30,31,30,31]

In [25]:
city_dynamic = task_df_copy['train'].groupby(['month_block', 'city_id']).item_cnt_day.sum().unstack(-1).fillna(0)
category_dynamic = task_df_copy['train'].groupby(['month_block', 'global_item_category_name_id']).item_cnt_day.sum().unstack(-1).fillna(0)

In [26]:
thresh = np.percentile(city_dynamic[2], 75)
series__ = tr.diff(city_dynamic[2][city_dynamic[2] > thresh].reset_index(), 1).month_block
# city_dynamic[2][city_dynamic[2] > thresh].reset_index().month_block.to_list()[-1]
series__[series__ > 1].mean()

8.5

In [27]:
def decode_pair(_id):
    city = idx.iloc[_id]['city_id']
    category = idx.iloc[_id]['global_item_category_name_id']
    return city, category


In [28]:
def generate_feature_row_base(split_x, target, window, _id):
    
    city_id, category_id = decode_pair(_id)

    _city_dynamic = city_dynamic[city_id]
    _category_dynamic = category_dynamic[category_id]

    _feature_set = {}
    _serieses = {}


    _feature_set['window'] = window
    _feature_set['id'] = _id

    _sale_series = split_x.set_index('month_block').item_cnt_month

    thresh_75 = np.percentile(_sale_series, 70)
    mean_75 = tr.diff(_sale_series[_sale_series >= thresh_75].reset_index(), 1).month_block
    mean_75 = mean_75[mean_75 > 1].mean()
    last_75 = _sale_series[_sale_series >= thresh_75].reset_index().month_block.to_list()[-1]
    
    thresh_90 = np.percentile(_sale_series, 90)
    mean_90 = tr.diff(_sale_series[_sale_series >= thresh_90].reset_index(), 1).month_block
    mean_90 = mean_90[mean_90 > 1].mean()
    last_90 = _sale_series[_sale_series >= thresh_90].reset_index().month_block.to_list()[-1]

    thresh_20 = np.percentile(_sale_series, 20)
    mean_20 = tr.diff(_sale_series[_sale_series <= thresh_20].reset_index(), 1).month_block
    mean_20 = mean_20[mean_20 > 1].mean()
    last_20 = _sale_series[_sale_series <= thresh_20].reset_index().month_block.to_list()[-1]


    _serieses['city_lag_1'] = tr.lag(_city_dynamic, 1, 0)
    _serieses['city_diff_1'] = tr.diff(_city_dynamic, 1)

    _serieses['category_lag_1'] = tr.lag(_category_dynamic, 1, 0)
    _serieses['category_diff_1'] = tr.diff(_category_dynamic, 1)

    _serieses['lag_1'] = tr.lag(_sale_series, 1, 0)
    _serieses['diff_1'] = tr.diff(_sale_series, 1)
    _serieses['diff_2'] = tr.diff(_sale_series, 2)

    custom_range = [1, 2, 3, 4, 6, 12]

    acf_main = acf(_sale_series, 13)

    for i in custom_range:
        _feature_set[f'lag_{i}_mul'] = _serieses['lag_1'].to_list()[-i]
        _feature_set[f'lag_{i}_mul_acf'] = _serieses['lag_1'].to_list()[-i] * acf_main[i] if _serieses['lag_1'].to_list()[-i] != 0 else 0
        _feature_set[f'city_lag_{i}'] = _serieses['city_lag_1'].to_list()[-i]
        _feature_set[f'category_lag_{i}'] = _serieses['category_lag_1'].to_list()[-i]

    for i in range(1,5):
        _feature_set[f'diff_1_{i}'] = _serieses['diff_1'].to_list()[-i]
        _feature_set[f'city_diff_1_{i}'] = _serieses['city_diff_1'].to_list()[-i]
        _feature_set[f'category_diff_1_{i}'] = _serieses['category_diff_1'].to_list()[-i]

    for i in range(1,5):
        _feature_set[f'diff_2_{i}'] = _serieses['diff_2'].to_list()[-i]
    
    add_aggregation(_feature_set, _sale_series, '', [np.min, np.max, np.mean, np.std])
    add_aggregation(_feature_set, _sale_series[-12:], 'last_12')
    add_aggregation(_feature_set, _sale_series[-6:], 'last_6')
    add_aggregation(_feature_set, _sale_series[-2:], 'last_2')
    add_aggregation(_feature_set, _serieses['diff_1'], 'diff_1')
    add_aggregation(_feature_set, _serieses['diff_2'], 'diff_2')
    add_aggregation(_feature_set, _serieses['diff_1'][-12:], 'diff_1_last_12', [np.min, np.max, np.mean, np.std])
    add_aggregation(_feature_set, _serieses['diff_1'][-6:], 'diff_1_last_6')
    add_aggregation(_feature_set, _serieses['diff_1'][-2:], 'diff_1_last_2')

    add_aggregation(_feature_set, _city_dynamic, 'city', [np.min, np.max, np.mean, np.std])
    add_aggregation(_feature_set, _category_dynamic, 'category', [np.min, np.max, np.mean, np.std])

    add_aggregation(_feature_set, _city_dynamic[-12:], 'city_last_12')
    add_aggregation(_feature_set, _city_dynamic[-2:], 'city_last_2')
    add_aggregation(_feature_set, _category_dynamic[-12:], 'category_last_12')
    add_aggregation(_feature_set, _category_dynamic[-2:], 'category_last_2')

    add_aggregation(_feature_set, _serieses['city_diff_1'][-3:], 'city_diff_1_last_3')
    add_aggregation(_feature_set, _serieses['category_diff_1'][-3:], 'category_diff_1_last_3')

    _feature_set['last_observed'] = _sale_series.values[-1]
    _feature_set['time'] = _sale_series.index[-1]
    _feature_set['month_number'] = _feature_set['time'] % 12
    _feature_set['days_in_month'] = DAYS[_feature_set['month_number']]
    _feature_set['target'] = target
    _feature_set['category_id'] = category_id
    _feature_set['city_id'] = city_id
    _feature_set['90_percentile'] = thresh_90 * (1 - last_90 / mean_90)
    _feature_set['20_percentile'] = thresh_20 * (1 - last_20 / mean_20)
    _feature_set['75_percentile'] = thresh_75 * (1 - last_75 / mean_75)


    return pd.DataFrame(_feature_set.items()).set_index(0).transpose()

In [29]:
train = None

if True:
    for window, split_x, split_y in iteration_rule(task_df['train']):
        as_list = split_y.item_cnt_month.to_list()
        for _id, subset in subset_extractor(split_x):
            new_feature = generate_feature_row_base(subset, as_list[_id], window, _id)
            if train is None:
                train = new_feature
            else:
                train = pd.concat([train, new_feature])
            
    train = train.reset_index().drop('index', axis=1)
    train.replace([np.inf, -np.inf], np.nan, inplace=True)
    train = train.fillna(0)
    load_pickle(train, 'extracted_train')
else:
    train = from_pickle('extracted_train.pkl')
target_test = task_df['test'].rename(columns={'month_block': 'window', 'item_cnt_month': 'target'})

NameError: name 'acf' is not defined

In [ ]:
uploader = DatasetUploader('')

In [ ]:
uploader.save(task_df['test'], 'full_dataset.csv')
uploader.save(task_df_copy['test'], 'extended_full_dataset.csv')
uploader.save(train, 'extracted_features.csv')
uploader.save(idx, 'idx.csv')